In [1]:
''' use base anaconda kernel '''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from PIL import Image

import time

from pathlib import Path

import cv2 as cv

import torch
import torchvision 

In [2]:
# loading in the data

# load in the dataframe:

df_cleaned = pd.read_csv('df_cleaned.csv')

df_cleaned.head()

/tmp/ipykernel_20956/2402431355.py:5: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cleaned = pd.read_csv('df_cleaned.csv')


,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo,x,y,w,h
0,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89537.0,car,False,True,False,836.812933,1014.734411,355.011547,418.198614,True,836.812933,355.011547,177.921478,63.187067
1,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89538.0,car,False,True,False,486.143187,705.727483,347.682421,422.725173,True,486.143187,347.682421,219.584296,75.042752
2,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89539.0,pedestrian,False,True,False,653.071594,670.531178,338.383372,357.505774,True,653.071594,338.383372,17.459584,19.122402
3,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89540.0,car,False,False,False,129.859655,198.354948,358.720910,406.558891,True,129.859655,358.720910,68.495292,47.837982
4,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89541.0,car,False,False,False,260.739030,324.526559,353.487298,400.046189,True,260.739030,353.487298,63.787529,46.558891


# Load in the train & test datasets

In [3]:
# load in the train & test datasets:

train_videos = pd.read_csv('train_videos.csv',header=None, index_col=0)
test_videos = pd.read_csv('test_videos.csv',header=None, index_col=0)

X_train_list = list(train_videos[1])
X_test_list = list(test_videos[1])


In [4]:
train_df = pd.read_csv('train_df.csv', index_col=0)
train_df.reset_index(inplace=True)
test_df = pd.read_csv('test_df.csv', index_col=0)
test_df.reset_index(inplace=True)


/tmp/ipykernel_20956/1855452194.py:1: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train_df.csv', index_col=0)
/tmp/ipykernel_20956/1855452194.py:3: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('test_df.csv', index_col=0)


In [5]:
train_df.head()

,name,videoName,frameIndex,id,category,attributes.crowd,attributes.occluded,attributes.truncated,box2d.x1,box2d.x2,box2d.y1,box2d.y2,haveVideo,x,y,w,h
0,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89537.0,car,False,True,False,836.812933,1014.734411,355.011547,418.198614,True,836.812933,355.011547,177.921478,63.187067
1,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89538.0,car,False,True,False,486.143187,705.727483,347.682421,422.725173,True,486.143187,347.682421,219.584296,75.042752
2,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89539.0,pedestrian,False,True,False,653.071594,670.531178,338.383372,357.505774,True,653.071594,338.383372,17.459584,19.122402
3,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89540.0,car,False,False,False,129.859655,198.354948,358.720910,406.558891,True,129.859655,358.720910,68.495292,47.837982
4,01c71072-718028b8-0000002.jpg,01c71072-718028b8,1,89541.0,car,False,False,False,260.739030,324.526559,353.487298,400.046189,True,260.739030,353.487298,63.787529,46.558891


In [6]:
# picking an example video:

print(f"Number of videos: {df_cleaned['videoName'].nunique()}")


video = df_cleaned['videoName'].unique()[0]
print(f"Video Name: {video}")

frameNums = df_cleaned[df_cleaned['videoName'] == video]['frameIndex'].unique()
numFrames = len(frameNums)

print(f"Number of frames for selected video: {numFrames}")

Number of videos: 961
Video Name: 01c71072-718028b8
Number of frames for selected video: 201


# Connect to AWS GPU

In [7]:
'''
CONNECT TO GPU OF AWS
'''

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print("GPU available.... connecting now")
    print(torch.cuda.get_device_name(0))  # Print GPU name
else:
    print("GPU not available")

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



GPU available.... connecting now
NVIDIA A10G


# Set up YOLO Model

In [8]:
# helper for easier paths to datasets

import os
HOME = os.getcwd()
print(HOME)

/home/ubuntu/project


In [10]:
# # Pip install method (recommended)

# !pip install git+https://github.com/ultralytics/ultralytics.git@main

# !pip install ultralytics

!pip install -r requirements.txt

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.11.5 torch-2.1.2 CUDA:0 (NVIDIA A10G, 22564MiB)
Setup complete ✅ (4 CPUs, 15.4 GB RAM, 105.2/248.1 GB disk)


In [11]:
'''OTHER POSSIBLE WAY OF INSTALLING YOLO WOULD HAVE BEEN USING CLONE 
(commented out below; we already installed w/ ultralytics above)'''

# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -e .

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

'OTHER POSSIBLE WAY OF INSTALLING YOLO WOULD HAVE BEEN USING CLONE \n(commented out below; we already installed w/ ultralytics above)'

In [12]:
# git clone https://github.com/ultralytics/ultralytics
# cd ultralytics
# pip install -e ultralytics

In [13]:
from ultralytics import YOLO

from IPython.display import display, Image

## 2 different methods:

- (1) CLI
    - task --> detect, classify, or segment 
    - mode --> train, val, predict, or export
    - model --> weights to use (yolov8n.pt)
    - conf --> confidence (0.25)
    - source --> image to use

- (2) SDK 



* test run a prediction on any image

In [47]:
# load in an example image to test it on:
video = train_df['name'][0]
print(video)

path = 'frames_v2/'
print(path)

01c71072-718028b8-0000002.jpg
frames_v2/


## CLI

* CLI is for trainnig, testing, and predicting in terminal

* testing it on external image:

In [46]:
# %cd {HOME}
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [45]:
# %cd {HOME}
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='frames_v2/01c71072-718028b8-0000002.jpg' save=True

In [44]:
# %cd {HOME}
# Image(filename=f'{HOME}01c71072-718028b8-0000002.jpg', width=600)

* trying it with our data

In [43]:
# %cd {HOME}
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source=video save=True

In [42]:
# %cd {HOME}
# Image(filename=video, height=600)

## SDK

* attempt on external image

In [19]:
model = YOLO(f'{HOME}/yolov8n.pt')
results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

100%|██████████| 104k/104k [00:00<00:00, 21.4MB/s]


image 1/1 /home/ubuntu/project/dog.jpeg: 640x384 1 person, 1 car, 1 dog, 103.5ms
Speed: 3.5ms preprocess, 103.5ms inference, 34.1ms postprocess per image at shape (1, 3, 640, 384)


In [20]:
results[0].boxes.xyxy

tensor([[   0.0000,  314.5087,  625.0571, 1278.1863],
        [  55.2816,  250.0128,  648.1115, 1266.2483],
        [ 633.2319,  719.5338,  701.0594,  786.0319]], device='cuda:0')

In [21]:
results[0].boxes.conf

tensor([0.7267, 0.2908, 0.2840], device='cuda:0')

In [22]:
results[0].boxes.cls

tensor([ 0., 16.,  2.], device='cuda:0')

* attempt on our own image

In [28]:
model = YOLO(f'{HOME}/yolov8n.pt')

results = model.predict(
   source='frames_v2/01c71072-718028b8-0000002.jpg',
   conf=0.25
)


image 1/1 /home/ubuntu/project/frames_v2/01c71072-718028b8-0000002.jpg: 384x640 3 cars, 5.8ms
Speed: 1.0ms preprocess, 5.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


In [29]:
results[0].boxes.xyxy

tensor([[  7.5780, 369.1844,  94.9579, 421.8085],
        [452.8893, 356.5795, 595.4589, 431.6505],
        [116.7452, 363.6737, 197.2686, 412.7683]], device='cuda:0')

In [30]:
results[0].boxes.conf

tensor([0.6936, 0.4158, 0.3331], device='cuda:0')

In [31]:
results[0].boxes.cls

tensor([2., 2., 2.], device='cuda:0')